# Thermal states for 3-SAT

In [1]:
import numpy as np
# import qit
import matplotlib
import matplotlib.pyplot as plt
from random import randint
# import pycosat
import random
import scipy.linalg as la
import pandas as pd
import scipy.sparse as sparse
import scipy.sparse.linalg as lasp
from numba import jit
# for parallel computations on CPU
import pymp
import qit
import scipy

from sat import *
from qaoa import *
from numpy import array

In [ ]:
%%time
N=20
beta=0.20
with pymp.Parallel(6) as pp:
    for m in pp.range(1, 20*N):
        clauses=[]
        overlap=[]
        for i in range(500):
            cnf=cnf_random_generator(N,m)
            overlap.append(thermal_overlap(cnf,N,beta))
            clauses.append(m)
        bits=[N]*500
        bet=[beta]*500
        df=pd.DataFrame(data={'bits':np.array(bits).ravel(), 
                            'beta':np.array(bet).ravel(), 
                            'clauses': np.array(clauses).ravel(), 
                            'overlap': np.array(overlap).ravel()})
        df=df[['bits','beta','clauses','overlap']]
        with open('3_SAT_cooling_20.csv', 'a') as f:
            df.to_csv(f, header=False)

# QAOA

In [2]:
B=B_operator(10)
initial_state=plus_state(10)

def optimization_function(H,p,params):
    ini_state=initial_state
    for k in range(p):
        ini_state=lasp.expm_multiply(1j*params[2*k]*H, ini_state)
        ini_state=lasp.expm_multiply(-1j*params[2*k+1]*B,ini_state)
    return ((sparse.spmatrix.getH(ini_state)).dot(H.dot(ini_state))).real, ini_state

def angles(H,p):
    def fun(x):
#         we minimize f to find max for F 
        return optimization_function(H, p, params=x)[0]
# starting point
    params_0=[0.25*np.pi for i in range(2*p)]
    params_min=[0 for i in range(2*p)]
    params_max=[2*np.pi if i%2==0 else np.pi for i in range(2*p)]
    # the bounds required by L-BFGS-B
    bounds = [(low, high) for low, high in zip(params_min, params_max)]
# use method L-BFGS-B because the problem is smooth and bounded
    result = scipy.optimize.minimize(fun, params_0, method="L-BFGS-B",bounds=bounds)
    return [result.x[i] for i in range(2*p)]



def optimal_p(cnf, N=10, e=0.75):
    H=cnf_to_hamiltonian(cnf,N)
    H_0=H.diagonal()
#     find minimal eigenvalue and the superposition of ground states
    min_energy=np.where(H_0 == H_0.min())[0]
#   mixinf of all ground states
    ground_state=np.zeros(2**N)
    for i in min_energy:
        ground_state[i]=1
    result=0
    p=0
    while result<1-e:
        p+=1
        ini_state=initial_state
        optimal_parameters=angles(H,p)
        qaoa_state=optimization_function(H,p,optimal_parameters)[1]
        for i in np.where(ground_state != 0)[0]:
            result+=np.abs(qaoa_state[i])**2

    return p

# return overlap between ground space and QAOA state
def overlap_p(p, cnf, N=10):
    H=cnf_to_hamiltonian(cnf,N)
    H_0=H.diagonal()
#     find minimal eigenvalue and the superposition of ground states
    min_energy=np.where(H_0 == H_0.min())[0]
#   mixinf of all ground states
    ground_state=np.zeros(2**N)
    for i in min_energy:
        ground_state[i]=1
    
    result=0
    optimal_parameters=angles(H,p)
    qaoa_state=optimization_function(H,p,optimal_parameters)[1]
    for i in np.where(ground_state != 0)[0]:
        result+=np.abs(qaoa_state[i])**2
        
    return result
    

In [ ]:
%%time
overlap_p(p=2,cnf=cnf_random_generator(10,42))

In [ ]:
%%time
optimal_p(cnf_random_generator(10,100))

In [ ]:
%%time
with pymp.Parallel(6) as pp:
    for m in pp.range(1, 100):
        clauses=[]
        iterations_p=[]
        for i in range(100):
            cnf=cnf_random_generator(10,m)
            iterations_p.append(optimal_p(cnf,10))
            clauses.append(m)
        df=pd.DataFrame(data={'p':np.array(iterations_p).ravel(), 
                            'clauses': np.array(clauses).ravel(), 
                            })
        df=df[['p','clauses']]
        with open('3_SAT_qaoa_10_025.csv', 'a') as f:
            df.to_csv(f, header=False)

## fix p

In [ ]:
%%time
p=2
with pymp.Parallel(6) as pp:
    for m in pp.range(1, 100):
        clauses=[]
        overlap=[]
        for i in range(500):
            cnf=cnf_random_generator(10,m)
            overlap.append(overlap_p(p,cnf,10))
            clauses.append(m)
        iterations_p=[p]*500
        df=pd.DataFrame(data={'p':np.array(iterations_p).ravel(), 
                            'clauses': np.array(clauses).ravel(),
                            'overlap': np.array(overlap).ravel(),
                            })
#         df=df[['p','clauses','overlap']]
        with open('3_SAT_qaoa_10_fix_p.csv', 'a') as f:
            df.to_csv(f, header=False)